<a href="https://colab.research.google.com/github/hmarinoo/DIO_Santander_Bootcamp_2025/blob/main/Santander_Bootcamp_2025_SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [14]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [15]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
user_nomes = df['name'].tolist()
print('ids:' ,user_ids,' nomes' ,user_nomes)

ids: [1, 2, 3, 4]  nomes ['Jophy', 'mothy', 'raphyn', 'minphy']


In [28]:
#import requests
#import json
users = df.to_dict(orient='records')
print(users)
#def get_user(id):
#  response = requests.get(f'{sdw2023_api_url}/users/{id}')
#  return response.json() if response.status_code == 200 else None

# user_ids comes from the previous cell NYydpX_GLRCB
#users = [user for id in user_ids if (user := get_user(id)) is not None]
# print(json.dumps(users, indent=2))

[{'UserID': 1, 'name': 'Jophy'}, {'UserID': 2, 'name': 'mothy'}, {'UserID': 3, 'name': 'raphyn'}, {'UserID': 4, 'name': 'minphy'}]


## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [17]:
!pip install openai

In [18]:
#import requests
for user  in users:
  print(user['name'])

# user = {id:users[1,'UserId'], name:users[1,'name']}
# print(user)
#
# import json

# def get_user(id):
#  response = requests.get(f'{sdw2023_api_url}/users/{id}')
#  return response.json() if response.status_code == 200 else None

#users = [user for id in user_ids if (user := get_user(id)) is not None]
#print(json.dumps(users, indent=2))

Jophy
mothy
raphyn
minphy


In [20]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'APIKey'

In [29]:
import openai

# Initialize the OpenAI client with your API key
client = openai.OpenAI(api_key=openai_api_key)

def generate_ai_news(user):
  completion = client.chat.completions.create(
    model="gpt-5-nano", # Changed model from gpt-4 to gpt-3.5-turbo
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos e segerindo investimentos mais seguros (máximo de 200 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('"')

# Convert the DataFrame to a list of dictionaries for easier iteration
# users_data = users.to_dict('records') # This line is commented out as 'users' is already a list of dicts

for user_dict in users:
  news = generate_ai_news(user_dict)
  print(news)
  # The 'news' key might not exist initially, or might not be a list
  if 'news' not in user_dict or not isinstance(user_dict.get('news'), list):
      user_dict['news'] = []
  user_dict['news'].append({
            "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Jophy, investir é cuidar do futuro. Opte por segurança: Tesouro Selic, CDBs, LCIs/LCAs. Diversifique, acompanhe o rendimento e invista de forma constante.
Mothy, investir é proteger seu futuro. Priorize segurança: diversifique em renda fixa (Tesouro Direto, CDBs) e planeje seus objetivos. Comece hoje.
Raphyn, investir é abrir portas pro futuro. Priorize segurança: Tesouro Direto, CDBs e fundos de curto prazo. Diversifique para proteger seu capital.
Minphy: investir é cuidar do futuro. Priorize segurança: Tesouro Direto, CDB/LCI/LCAs e renda fixa DI. Planeje, diversifique e tenha tranquilidade.


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [31]:
pd.DataFrame(users).to_csv('users_news.csv')
#